<img src="../../static/aeropython_name_mini.png" alt="AeroPython" style="width: 300px;"/>

## Interactuar con arduino desde un Notebook

In [ ]:
import serial
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def deltatime():
    tiempo = datetime.datetime.now()
    deltat = tiempo - starttime
    return deltat.seconds + deltat.microseconds/1000000

In [ ]:
def decode_medida(lectura):
    _pos = lectura.find('posicion') + 10
    _pos2 = lectura.find(' ',_pos )
    medida_1 = float(lectura[_pos : _pos2])

    _pos = lectura.find('velocidad') + 11
    _pos2 = lectura.find(' ',_pos )
    medida_2 = float(lectura[_pos : _pos2])
    
    return (medida_1, medida_2)

In [ ]:
def com_status(tiempos, posiciones, velocidades, hitos, serial_com):
    serial_com.write(b'status')
    lectura = serial_com.readline().decode("utf-8")
    print(lectura) #Espera hasta que empiece a comunicarse
    if lectura[:22] == 'Modo escucha. Medida: ':
        tiempos.append(deltatime())
        medida = float(lectura[22:])
        posiciones.append(medida)
        velocidades.append(0.0)

In [ ]:
def com_start_full():
    ser = serial.Serial('COM4', 9600) #Iniciamos comunicación
    #Linux: ser = serial.Serial('/dev/ttyACM0', 9600)
    ser.readline()
    ser.write(b'calibrate')
    ser.readline()
    starttime = datetime.datetime.now()
    return ser, starttime

In [ ]:
def com_move(tiempos, posiciones, velocidades, hitos, serial_com, destino, medida):
    destino = str(destino).encode()
    serial_com.write(b'move')
    serial_com.readline()
    serial_com.write(destino) #Enviamos una orden
    print('mensaje enviado\r\n')
    posiciones.append(medida)
    velocidades.append(0.0)       
    tiempos.append(deltatime())
    #hitos.append(deltatime())

In [ ]:
def com_read_move(tiempos, posiciones, velocidades, hitos, serial_com):
    finish = False
    lectura = serial_com.readline().decode("utf-8")
    print(lectura) 
    if lectura == 'Alcanzado\r\n':
        hitos.append(deltatime())
        finish = True
    if lectura[:8] == 'moviendo':
        medida = decode_medida(lectura)
        posiciones.append(medida[0])
        velocidades.append(medida[1])        
        tiempos.append(deltatime())
    return finish

In [ ]:
tiempos = []
posiciones =[]
velocidades = []
hitos = []
destinos = [15,60,90,120,10]

ser, starttime = com_start_full()

print('preparado')
for destino in destinos:
    # -------------------------------
    com_status(tiempos, posiciones, velocidades, hitos, ser)
    # -----------------------------------

    com_move(tiempos, posiciones, velocidades, hitos, ser, destino, posiciones[-1])

    for i in range(100): #Leemos hasta que pase a 'Esperando', o hasta que pasen 100 pasos
        finish = com_read_move(tiempos, posiciones, velocidades, hitos, ser)
        if finish: break
            
ser.close()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(tiempos, posiciones, label = 'Posición')
plt.plot(tiempos, np.array(velocidades)*100, label = 'Velocidad (x100)')
plt.hlines(0, 0, tiempos[-1], 'k','--')
for hito in hitos:
    plt.vlines(hito, -50,170, 'r', '--')
plt.legend(loc = 2)

In [ ]:
tiempos

In [ ]:
posiciones